# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Import</p></div>

In [ ]:
!pip install -q -U autolgbm

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import gc
gc.enable()

from autolgbm import AutoLGBM
# Warning
import warnings
warnings.filterwarnings('ignore')

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Configuration</p></div>

In [ ]:
BASE_PATH = "../input/mens-march-mania-2022/MDataFiles_Stage2/"
RESULTS_NAME = "MNCAATourneyDetailedResults.csv"
SEED_NAME = "MNCAATourneySeeds.csv"
RANK_NAME = "MMasseyOrdinals_thruDay128.csv"
TEST_NAME = "MSampleSubmissionStage2.csv"
# MODEL
train_filename = "./train.csv"
output = "output"
test_filename = "./test.csv"
task = None
targets = ['target']
features_model = None
categorical_features = None
use_gpu = False
num_folds = 20
seed = 42
num_trials = 10000
time_limit = 7200
fast = False


#  <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Overview</p></div>

I wrote another kernel to Exploratory Data Analysis in [here](https://www.kaggle.com/phanttan/basiceda-2022marchmania)
Reading it to more understand dataset in this competition.

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Dataset</p></div>
****
**Stage 1** - You should submit predicted probabilities for every possible matchup in the past 5 NCAA® tournaments (2016-2019 and 2021). Note that there was no tournament held in 2020.

**Stage 2** - You should submit predicted probabilities for every possible matchup before the 2022 tournament begins.

Refer to the Timeline page for specific dates. In both stages, the sample submission will tell you which games to predict.

In [ ]:
!ls ../input/mens-march-mania-2022/MDataFiles_Stage2/

   # <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">Training Dataset</p></div>

In [ ]:
TourneyDetailedResults = pd.read_csv(BASE_PATH + RESULTS_NAME)
TourneyDetailedResults.head()

   # <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:70%;text-align:left">Win or Lose</p></div>

In [ ]:
box_col = ["FGM","FGA","FGM3","FTM","FTA","OR","DR","Ast","TO","Stl","Blk","PF"]
# Win Team
df_boxW = TourneyDetailedResults[["Season","WTeamID"]+["W" + col for col in box_col]]
df_boxW = df_boxW.rename(columns={"WTeamID":"TeamID"})
df_boxW = df_boxW.rename(columns={("W"+ col):col for col in box_col})
# Loss Team
df_boxL = TourneyDetailedResults[["Season", "LTeamID"]+["L" + col for col in box_col]]
df_boxL = df_boxL.rename(columns={"LTeamID":"TeamID"})
df_boxL = df_boxL.rename(columns={("L"+col):col for col in box_col})

# # Merge Teams
df_box = pd.merge(df_boxW, df_boxL, on=["Season","TeamID"]+box_col, how="outer")
df_box.head()
            

   # <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:70%;text-align:left">Define Target Feature</p></div>

In [ ]:
TourneyDetailedResults2 = TourneyDetailedResults
TourneyDetailedResults = TourneyDetailedResults.rename(columns={"WTeamID":"Team1ID","LTeamID":"Team2ID","WScore":"T1Score","LScore":"T2Score"})
TourneyDetailedResults = TourneyDetailedResults.rename(columns={f"W{col}":f"T1{col}" for col in box_col})
TourneyDetailedResults = TourneyDetailedResults.rename(columns={f"L{col}":f"T2{col}" for col in box_col})
TourneyDetailedResults2 = TourneyDetailedResults2.rename(columns={"WTeamID":"Team2ID","LTeamID":"Team1ID","WScore":"T2Score","LScore":"T1Score"})

In [ ]:
features = ["Season","Team1ID","Team2ID","T1Score","T2Score",'target']
# If Team 1 wins, target = 1 
TourneyDetailedResults['target'] = 1.0
# If Team 1 lost, target = 0
TourneyDetailedResults2['target'] = 0.0
train = pd.merge(TourneyDetailedResults,TourneyDetailedResults2,on = features,how="outer")
train = train[features]
train.head()

In [ ]:
box_T1 = df_box.copy()
box_T2 = df_box.copy()
box_T1.columns = ['Season','Team1ID'] + ["T1"+col+"_mean" for col in box_col]
box_T2.columns = ['Season','Team2ID'] + ["T2"+col+"_mean" for col in box_col]
train = pd.merge(train,box_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,box_T2,on = ["Season","Team2ID"],how = "left")
train.head()

   # <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:70%;text-align:left">Seed As Feature</p></div>

In [ ]:
df_seeds = pd.read_csv(BASE_PATH + SEED_NAME)
df_seeds.head()

In [ ]:
df_seeds["seed"] = df_seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds_T1 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T2 = df_seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','Team1ID','T1_seed']
seeds_T2.columns = ['Season','Team2ID','T2_seed']
train = pd.merge(train,seeds_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,seeds_T2,on = ["Season","Team2ID"],how = "left")
train["seeddiff"] = train["T1_seed"] - train["T2_seed"]

   # <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:70%;text-align:left">Ranking As Feature</p></div>

In [ ]:
MMOriginals = pd.read_csv(BASE_PATH + RANK_NAME)
df_rank = MMOriginals.groupby(["Season","TeamID"])["OrdinalRank"].agg(np.mean).reset_index()
df_rank.head()

In [ ]:
ranks_T1 = df_rank.copy()
ranks_T2 = df_rank.copy()
ranks_T1.columns = ['Season','Team1ID','T1_rank_mean']
ranks_T2.columns = ['Season','Team2ID','T2_rank_mean']
train = pd.merge(train,ranks_T1,on = ["Season","Team1ID"],how = "left")
train = pd.merge(train,ranks_T2,on = ["Season","Team2ID"],how = "left")
train["rankdiff"] = train["T1_rank_mean"] - train["T2_rank_mean"]

In [ ]:
train = train.drop(columns = ["T1Score","T2Score"])

In [ ]:
train.head()

   # <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:70%;text-align:center">Testing Dataset</p></div>

In [ ]:
test = pd.read_csv(BASE_PATH + TEST_NAME)
test.head()

In [ ]:
test["Season"] = test['ID'].apply(lambda x: int(x[0:4]))
test["Team1ID"] = test['ID'].apply(lambda x: int(x[5:9]))
test["Team2ID"] = test['ID'].apply(lambda x: int(x[10:14]))
test.head()

In [ ]:
test = pd.merge(test,box_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,box_T2,on = ["Season","Team2ID"],how = "left")
test.head()

In [ ]:
test = pd.merge(test,seeds_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,seeds_T2,on = ["Season","Team2ID"],how = "left")
test["seeddiff"] = test["T1_seed"] - test["T2_seed"]
test.head()

In [ ]:
test = pd.merge(test,ranks_T1,on = ["Season","Team1ID"],how = "left")
test = pd.merge(test,ranks_T2,on = ["Season","Team2ID"],how = "left")
test["rankdiff"] = test["T1_rank_mean"] - test["T2_rank_mean"]
test.drop(columns=['ID','Pred'])
test.head()

In [ ]:
train.to_csv("train.csv",index = None)
test.to_csv("test.csv",index = None)

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">MODEL and TRAINING</p></div>


In [ ]:
algbm = AutoLGBM(
    train_filename=train_filename,
    output=output,
    test_filename=test_filename,
    task=task,
    targets=targets,
    features=features_model,
    categorical_features=categorical_features,
    use_gpu=use_gpu,
    num_folds=num_folds,
    seed=seed,
    num_trials=num_trials,
    time_limit=time_limit,
    fast=fast,
)

In [ ]:
algbm.train()

# <div style="color:white;display:fill;border-radius:5px;background-color:#75B7BF;letter-spacing:0.1px;overflow:hidden"><p style="padding:20px;color:white;overflow:hidden;margin:0;font-size:100%;text-align:center">INFERENCE</p></div>


In [ ]:
submission = pd.read_csv("../input/mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv")
autolgb_pred = pd.read_csv("./output/test_predictions.csv")
submission['Pred'] = autolgb_pred['1.0']
# submission.rename(columns = {'1.0':'Pred'}, inplace = True)
submission.to_csv("submission.csv", index=False)
submission.head()